In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

ModuleNotFoundError: No module named 'torch'

In [57]:
fileName = 'result'

In [58]:
df = pd.read_excel(rf'.\dataset\combine\{fileName}.xlsx')
df

,index,거래량,거래대금,date,시가,고가,저가,isUp,positive,negative,스포츠,사회,정치,경제,생활/문화,IT/과학
0,0,0,0,20210812,55200,55200,55200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,224755,12500,20210811,55800,56900,54500,0,0.116271,0.883729,1.19,18.33,0.32,46.94,69.95,3.63
2,2,224755,12500,20210811,55800,56900,54500,0,0.752298,0.247702,3.50,6.31,0.73,1.25,74.07,59.50
3,3,224755,12500,20210811,55800,56900,54500,0,0.086373,0.913627,30.83,12.35,6.10,48.09,45.13,30.53
4,4,224755,12500,20210811,55800,56900,54500,0,0.174077,0.825923,10.28,17.41,2.84,18.00,63.87,75.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,1449,697145,24860,20190315,36250,36400,35150,0,0.006811,0.993189,8.08,11.77,2.41,9.10,20.71,0.55
1450,1450,697145,24860,20190315,36250,36400,35150,0,0.034088,0.965912,78.29,19.07,4.51,89.26,38.70,5.70
1451,1451,697145,24860,20190315,36250,36400,35150,0,0.014838,0.985162,61.55,22.94,3.29,13.18,49.77,14.13
1452,1452,697145,24860,20190315,36250,36400,35150,0,0.027283,0.972717,75.00,9.52,3.57,25.70,37.96,7.66


In [59]:
df.isnull().sum()   # 결측치 확인

index         0
거래량           0
거래대금          0
date          0
시가            0
고가            0
저가            0
isUp          0
positive    224
negative    224
스포츠         224
사회          224
정치          224
경제          224
생활/문화       224
IT/과학       224
dtype: int64

In [60]:
df = df.fillna(method='bfill')  #결측치 전날 값으로 채우기
df.isnull().sum()

index       0
거래량         0
거래대금        0
date        0
시가          0
고가          0
저가          0
isUp        0
positive    0
negative    0
스포츠         0
사회          0
정치          0
경제          0
생활/문화       0
IT/과학       0
dtype: int64

In [62]:
df.duplicated().sum()   # 중복값 확인

0

In [66]:
class LSTM1(nn.Module): 
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length): 
        super(LSTM1, self).__init__() 
        self.num_classes = num_classes # number of classes 
        self.num_layers = num_layers # number of layers 
        self.input_size = input_size # input size 
        self.hidden_size = hidden_size # hidden state 
        self.seq_length = seq_length # sequence length 
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True) # lstm 
        self.fc_1 = nn.Linear(hidden_size, 128) # fully connected 1 
        self.fc = nn.Linear(128, num_classes) # fully connected last layer 
        self.relu = nn.ReLU() 
        
    def forward(self,x): 
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) # hidden state 
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) # internal state 
                                                                                             # Propagate input through LSTM 
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # lstm with input, hidden, and internal state 
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next 
        out = self.relu(hn) 
        out = self.fc_1(out) # first Dense 
        out = self.relu(out) # relu 
        out = self.fc(out) # Final Output 
        return out


In [64]:
X = df[df.columns.difference(['isUp'])]
y = df['isUp'].astype(int)

from sklearn.preprocessing import StandardScaler, MinMaxScaler 
mm = MinMaxScaler() 
ss = StandardScaler() 
X_ss = ss.fit_transform(X) 
X_mm = mm.fit_transform(X)
y = np.array([y.values]).T  # Torch로 만들기 위해 array로 변환

In [70]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # GPU setting

kfold = KFold(n_splits=5)
cv_accuracy = []
n_iter = 0

scaler = [X_ss, X_mm]

for X_scaled in scaler:
    for train_idx, test_idx in kfold.split(X_scaled):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx] 
                        
        X_train_tensors = Variable(torch.Tensor(X_train)) # array를 torch로 변환
        X_test_tensors = Variable(torch.Tensor(X_test)) 
        y_train_tensors = Variable(torch.Tensor(y_train)) 
        y_test_tensors = Variable(torch.Tensor(y_test)) 
        X_train_tensors_final = torch.reshape(X_train_tensors, (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))
        X_test_tensors_final = torch.reshape(X_test_tensors, (X_test_tensors.shape[0], 1, X_test_tensors.shape[1]))

        # 네트워크 parameter 구성
        num_epochs = 3 #0000 #1000 epochs
        learning_rate = 0.00001 #0.001 lr

        input_size = 15 #number of features
        hidden_size = 2 #number of features in hidden state
        num_layers = 1 #number of stacked lstm layers

        num_classes = 1 #number of output classes 
        lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]).to(device)

        loss_function = torch.nn.MSELoss()    # mean-squared error for regression
        optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)  # adam optimizer
        
        for epoch in range(num_epochs): 
            outputs = lstm1.forward(X_train_tensors_final.to(device)) # forward pass 
            optimizer.zero_grad() # caluclate the gradient, manually setting to 0 
            
            # obtain the loss function 
            loss = loss_function(outputs, y_train_tensors.to(device)) 
            loss.backward() # calculates the loss of the loss function 
            optimizer.step() # improve from loss, i.e backprop 
            if epoch % 100 == 0: 
                print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
        
        # 예측
        
        train_predict = lstm1(X_test_tensors_final.to(device))#forward pass
        data_predict = train_predict.data.detach().cpu().numpy() #numpy conversion
                
        print(data_predict)
        # 정확도 측정
        n_iter += 1
        accuracy = np.round(accuracy_score(y_test, data_predict), 4)
        print(f'\n{n_iter} 교차검증 정확도 : {accuracy} , 학습 데이터 크기 : {X_train.shape[0]} , 검증 데이터 크기 : {X_test.shape[0]}')
        cv_accuracy.append(accuracy)

    print('\n평균검증 정확도 :  ', np.mean(cv_accuracy))

Epoch: 0, loss: 0.41972
<class 'numpy.ndarray'>
[[0.05953975]
 [0.05707597]
 [0.06535032]
 [0.06111491]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06132728]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06533129]
 [0.06420363]
 [0.06458493]
 [0.06455992]
 [0.06473079]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.10450735]
 [0.06535032]
 [0.06614926]
 [0.06115667]
 [0.10945775]
 [0.16132063]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.09415315]
 [0.05496052]
 [0.09002915]
 [0.06483839]
 [0.0643903 ]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06071255]
 [0.05931634]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06369114]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06463365]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.06535032]
 [0.08777479]
 [0.09381124]
 [0.0620933 ]
 [0.06310022]
 [0.06525013]
 [0.06535032]


ValueError: Classification metrics can't handle a mix of binary and continuous targets